# Anatomy of ring and motor-related neurons in the hindbrain

In [ ]:
%matplotlib widget
# %gui qt
from pathlib import Path
from shutil import copy

import flammkuchen as fl
import napari
import numpy as np
import pandas as pd
from lotr import LotrExperiment, dataset_folders
from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
i = 0
all_centroids = dict()

In [ ]:
path = dataset_folders[i]
# for path in dataset_folders[:2]:
exp = LotrExperiment(path)
v = napari.Viewer()
anatomy_stack = exp.anatomy_stack
roi_stack = exp.rois_stack + 1

v.add_image(anatomy_stack, contrast_limits=(0, 30))
v.add_labels(roi_stack)
pts = v.add_points([], ndim=3)

In [ ]:
all_centroids[path.name] = pts.data
i += 1
v.close()

In [ ]:
all_centroids

In [ ]:
fl.save("/Users/luigipetrucco/Desktop/centroids.h5", all_centroids)

In [ ]:
for path, (name, vals) in zip(dataset_folders, all_centroids.items()):
    assert path.name == name

    with open(path / "centroid.txt", "w") as f:
        f.writelines([f"{vals[0, i]}\n" for i in [0, 2, 1]])

In [ ]:
data_df = []
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)
    coords = exp.coords_um
    cent_coords = exp.morphed_coords_um

    data_dict = {f"c{i}": coords[:, i] for i in range(3)}
    data_dict.update({f"centered{i}": cent_coords[:, i] for i in range(3)})
    data_dict["fid"] = path.name

    data_df.append(pd.DataFrame(data_dict))

data_df = pd.concat(data_df, ignore_index=1)

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(7, 3))
axs[0].scatter(
    data_df["c1"], data_df["c2"], lw=0, s=5, alpha=0.03, color=(0.3,) * 3,
)
axs[1].scatter(
    data_df["c1"], -data_df["c0"] * 10, lw=0, s=5, alpha=0.03, color=(0.3,) * 3,
)
for i in range(2):
    # axs[i].set_title(k)
    axs[i].axis("equal")
    axs[i].axis("off")